In [1]:
%load_ext autoreload
%autoreload 2
from cyecca import lie
import casadi as ca
import numpy as np
import scipy.linalg

In [2]:
np.set_printoptions(precision=3, linewidth=100)

In [3]:
x = lie.se23.elem(ca.vertcat(0.1,0.2,0.3,0.4,0.5,0.6,0.1,0.2,0.3))

$e^{ad_x} = Ad e^x$

In [4]:
exp_ad_x = scipy.linalg.expm(ca.DM(x.ad()))
exp_ad_x

array([[ 0.936, -0.283,  0.21 ,  0.   ,  0.   ,  0.   , -0.273, -0.491,  0.556],
       [ 0.303,  0.951, -0.068,  0.   ,  0.   ,  0.   ,  0.618, -0.215, -0.248],
       [-0.181,  0.127,  0.975,  0.   ,  0.   ,  0.   , -0.379,  0.512, -0.137],
       [ 0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 , -0.127, -0.26 ,  0.215],
       [ 0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068,  0.299, -0.098, -0.034],
       [ 0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975, -0.157,  0.152, -0.049],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975]])

In [5]:
Ad_exp_x = np.array(ca.DM(x.exp(lie.SE23Mrp).Ad()))
Ad_exp_x

array([[ 0.936, -0.283,  0.21 ,  0.   ,  0.   ,  0.   , -0.273, -0.491,  0.556],
       [ 0.303,  0.951, -0.068,  0.   ,  0.   ,  0.   ,  0.618, -0.215, -0.248],
       [-0.181,  0.127,  0.975,  0.   ,  0.   ,  0.   , -0.379,  0.512, -0.137],
       [ 0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 , -0.127, -0.26 ,  0.215],
       [ 0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068,  0.299, -0.098, -0.034],
       [ 0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975, -0.157,  0.152, -0.049],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.936, -0.283,  0.21 ],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.303,  0.951, -0.068],
       [ 0.   ,  0.   ,  0.   ,  0.   ,  0.   ,  0.   , -0.181,  0.127,  0.975]])

In [6]:
np.linalg.norm(exp_ad_x - Ad_exp_x)

2.6193750069204537e-16

In [8]:
x.algebra.elem(ca.SX.sym("x", 9)).ad()

SX(@1=(-x_7), @2=(-x_8), @3=(-x_6), 
[[00, @2, x_7, 00, 00, 00, 00, (-x_5), x_4], 
 [x_8, 00, @3, 00, 00, 00, x_5, 00, (-x_3)], 
 [@1, x_6, 00, 00, 00, 00, (-x_4), x_3, 00], 
 [00, 00, 00, 00, @2, x_7, 00, (-x_2), x_1], 
 [00, 00, 00, x_8, 00, @3, x_2, 00, (-x_0)], 
 [00, 00, 00, @1, x_6, 00, (-x_1), x_0, 00], 
 [00, 00, 00, 00, 00, 00, 00, @2, x_7], 
 [00, 00, 00, 00, 00, 00, x_8, 00, @3], 
 [00, 00, 00, 00, 00, 00, @1, x_6, 00]])

In [9]:
x.diff_correction_inv()

SX(@1=0.978484, @2=0.151568, @3=-0.0938736, @4=-0.144948, @5=0.98345, @6=0.0593496, @7=0.103804, @8=-0.0394891, @9=0.991725, 
[[@1, @4, @7, 00, 00, 00, -0.0919934, -0.267181, 0.271445], 
 [@2, @5, @8, 00, 00, 00, 0.31, -0.0722919, -0.150667], 
 [@3, @6, @9, 00, 00, 00, -0.212181, 0.239404, -0.0460762], 
 [00, 00, 00, @1, @4, @7, -0.0427297, -0.138217, 0.106388], 
 [00, 00, 00, @2, @5, @8, 0.151364, -0.032869, -0.0285422], 
 [00, 00, 00, @3, @6, @9, -0.0866664, 0.067985, -0.0164345], 
 [00, 00, 00, 00, 00, 00, @1, @4, @7], 
 [00, 00, 00, 00, 00, 00, @2, @5, @8], 
 [00, 00, 00, 00, 00, 00, @3, @6, @9]])

In [10]:
import math

In [11]:
def derive_Ul_inv_series(n):
    x = lie.se23.elem(ca.SX.sym('x', 9))
    X = x.ad()
    Ul_inv = ca.SX.zeros(9, 9)
    #print(ca.mpower(X, 0))
    for k in range(n):
        Xp = ca.mpower(X, k)
        #print(k)
        #print(Xp.sparsity().spy())
        Ul_inv += Xp/math.factorial(k+1)
    Ul_inv = ca.sparsify(Ul_inv)
    return ca.Function('Ul_inv_series', [x.param], [Ul_inv])
Ul_inv_series = derive_Ul_inv_series(10)

In [12]:
np.set_printoptions(precision=5, linewidth=200, suppress=False)
np.array(ca.DM(x.diff_correction_inv()) - ca.DM(Ul_inv_series(x.param)))

array([[-1.24989e-12,  2.25903e-13,  2.66065e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00, -2.82378e-11,  5.60102e-12,  6.43652e-12],
       [ 1.58734e-13, -9.61453e-13,  5.88092e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  3.93069e-12, -2.18101e-11,  1.34444e-11],
       [ 3.10835e-13,  5.65707e-13, -4.80838e-13,  0.00000e+00,  0.00000e+00,  0.00000e+00,  7.57250e-12,  1.28431e-11, -1.14820e-11],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00, -1.24989e-12,  2.25903e-13,  2.66065e-13, -1.24983e-11,  2.29233e-12,  2.63782e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  1.58734e-13, -9.61453e-13,  5.88092e-13,  1.55320e-12, -9.61405e-12,  5.89162e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  3.10835e-13,  5.65707e-13, -4.80838e-13,  3.13059e-12,  5.64523e-12, -4.80702e-12],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00000e+00, -1.24989e-12,  2.25903e-13,  2.66065e-13],
       [ 0.00000e+00,  0.00000e+00,  0.00000e+00,  0.00

In [15]:
%%timeit
x.diff_correction_inv

30 ns ± 0.0387 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [14]:
%%timeit
Ul_inv_series(x.param)

889 µs ± 1.23 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
